# INITIALIZING

In [ ]:
import os
import csv
import copy
import math
import random
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize, scale
from torch.optim import lr_scheduler
from tqdm import tqdm
import torch.nn.utils.prune as prune

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))

# Random seed setup
random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Hyperparameters
n_split = 10
num_epochs = 60
weight_decay = 0
hidden_weights_pruning = True
hidden_weights_pruning_ratio = 0.9
early_stop = 50
batch_size = 45
lr = 0.001
embedding_size = 200
embedding_size_last = 320
num_blocks = 4
n_head = 4
using_cancer_type = True
scale_y = True
using_tf_gene_matrix = True
trim_ratio = 0.6
top_k = math.floor(embedding_size * trim_ratio)
top_k_last_layer = math.floor(embedding_size_last * trim_ratio)
embed_pretrain_gene2vec = True
permutation_sga_tissue = False


Using device: cuda
NVIDIA GeForce RTX 4050 Laptop GPU


# Setting up Data

In [ ]:
# Load required datasets
tf_gene_df = pd.read_csv("Data/CITRUS_tf_gene.csv", index_col=0) if using_tf_gene_matrix else None
tcga_sga_df = pd.read_csv("Data/TCGA_sga_data.csv", index_col=0)
tcga_RNAseq_df = pd.read_csv("Data/TCGA_RNAseq_data.csv", index_col=0)
tcga_can_df = pd.read_csv("Data/TCGA_xena_map_cancertype.csv", index_col=0)
gdsc_sga_df = pd.read_csv("Data/GDSC_sga_data.csv", index_col=0)
gdsc_RNAseq_df = pd.read_csv("Data/GDSC_RNAseq_data.csv", index_col=0)
gdsc_can_df = pd.read_csv("Data/GDSC_cancer_type.csv", index_col=0)

# Data cleaning and alignment
tcga_RNAseq_df[tcga_RNAseq_df < 0] = 0
gdsc_RNAseq_df[gdsc_RNAseq_df < 0] = 0
tcga_RNAseq_df = tcga_RNAseq_df.loc[tcga_sga_df.index, :]
tcga_can_df = tcga_can_df.loc[tcga_sga_df.index, ["cancer_type"]]
gdsc_RNAseq_df = gdsc_RNAseq_df.loc[gdsc_sga_df.index, :]
gdsc_can_df = gdsc_can_df.loc[gdsc_sga_df.index, :]
tf_gene_df[tf_gene_df>0]=1
tf_gene_gene_list = tf_gene_df.columns

# Combine TCGA and GDSC datasets
sga_df = pd.concat([tcga_sga_df, gdsc_sga_df])
deg_df = pd.concat([tcga_RNAseq_df, gdsc_RNAseq_df])
can_df = pd.concat([tcga_can_df, gdsc_can_df])

gene_exprs_list = pd.read_csv("Data/exprs_gene_list_GDSC_yifan_mike.txt", header=None)
gene_sga_list = pd.read_csv("Data/TCI_fondation1_driver_dataset_combine_gene_list.txt", header=None)

common_gene_sga = [gene for gene in sga_df.columns if gene in gene_sga_list.iloc[:, 0].tolist()]
common_gene_exprs = [gene for gene in deg_df.columns if gene in gene_exprs_list.iloc[:, 0].tolist()]
if using_tf_gene_matrix:
    common_gene_exprs = [gene for gene in common_gene_exprs if gene in tf_gene_df.columns]
    tf_gene_df = tf_gene_df.loc[:, common_gene_exprs]

sga_df = sga_df.loc[:, common_gene_sga]
deg_df = deg_df.loc[:, common_gene_exprs]

def map_cancer_type_to_id(df):
    mapping = {name: idx for idx, name in enumerate(df["cancer_type"].unique())}
    df["cancer_type"] = df["cancer_type"].map(mapping)
    return df, len(mapping)

can_df, can_size = map_cancer_type_to_id(can_df)

def sga_features_to_indices(sga_df):
    sga_feats = sga_df.to_numpy()   
    non_zero_idx = np.where(sga_feats != 0)
    sga_feats[non_zero_idx[0], non_zero_idx[1]] = np.array(non_zero_idx[1]) + 1  
    sga_df = pd.DataFrame(sga_feats, index=sga_df.index, columns=sga_df.columns) 
    return sga_df
sga_df = sga_features_to_indices(sga_df)
print(sga_df,deg_df)


                 B2M  SNX14  TET1  BCL2L1  HDAC1  RICTOR  GNGT2  MUC2  ROS1  \
TCGA-V1-A9ZK-01  0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
TCGA-55-8205-01  0.0    2.0   3.0     0.0    0.0     0.0    0.0   0.0   0.0   
TCGA-DR-A0ZM-01  1.0    0.0   0.0     0.0    0.0     6.0    0.0   0.0   0.0   
TCGA-HM-A3JK-01  0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
TCGA-KS-A4I1-01  0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
...              ...    ...   ...     ...    ...     ...    ...   ...   ...   
909779           0.0    0.0   0.0     0.0    0.0     6.0    0.0   0.0   0.0   
1240193          0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
1240168          0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   9.0   
753558           0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
753583           0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   

                  ALK  ...   AR   EPHA6  LETM2  KIT

# MODEL

In [ ]:
import torch
import torch.nn as nn
from gensim.models import Word2Vec
import pandas as pd
import numpy as np


class SGASelfAttn(nn.Module):
    """
    Implements a multi-head self-attention mechanism for SGA.
    Produces an embedding of dimension `embedding_dim` for a vector of SGA statuses.
    """

    def __init__(self, sga_gene_list, sga_size, embedding_dim, n_head, using_gene2vec_pretrain):
        super(SGASelfAttn, self).__init__()
        self.sga_gene_list = sga_gene_list
        self.sga_size = sga_size
        self.embedding_dim = embedding_dim
        self.n_head = n_head
        self.using_gene2vec_pretrain = using_gene2vec_pretrain

        # Initialize SGA embeddings
        if using_gene2vec_pretrain:
            gene2vec_model = Word2Vec.load(f"Processed Data/gene2vec/word2vec.model_{embedding_dim}")
            w2v_dict = {item: gene2vec_model.wv[item] for item in gene2vec_model.wv.key_to_index}
            gene2vec_df = pd.DataFrame.from_dict(w2v_dict)
            gene2vec_df = gene2vec_df.loc[:, self.sga_gene_list]
            embedding_matrix = np.insert(gene2vec_df.values, 0, np.zeros(gene2vec_df.shape[0]), axis=1)
            self.sga_embs = nn.Embedding.from_pretrained(torch.Tensor(embedding_matrix).T, padding_idx=0, freeze=False)
        else:
            self.sga_embs = nn.Embedding(num_embeddings=sga_size + 1, embedding_dim=embedding_dim, padding_idx=0)

        # Attention layers
        self.key_projection = nn.Linear(embedding_dim, embedding_dim)
        self.attention_projection = nn.Linear(embedding_dim, n_head)

    def forward(self, sga, mask=True):
        case_sga_embs = self.sga_embs(sga)  # B x I x E

        case_sga_keys = torch.tanh(self.key_projection(case_sga_embs))  # B x I x E

        case_sga_attn_heads = self.attention_projection(case_sga_keys)  # B x I x N
        
        if mask:
            sga_mask = sga.repeat(self.n_head, 1, 1).permute(1, 2, 0)
            case_sga_attn_heads = case_sga_attn_heads.masked_fill(sga_mask == 0, -1e9)

        # Softmax and weighted embedding
        A = torch.softmax(case_sga_attn_heads, dim=1)  # B x I x N
        emb_sga = torch.sum(torch.bmm(A.permute(0, 2, 1), case_sga_embs), dim=1)  # B x E

        return emb_sga, torch.sum(A, dim=2)


class ResGit(nn.Module):
    """
    Implements the GIT-RINN model with multiple self-attention blocks and optional pretraining.
    """

    def __init__(self, out_feat_size, sga_size, embedding_dim, embedding_dim_last, can_size=10, n_head=5,
                 num_attn_blocks=3, using_cancer_type=True, using_tf_gene_matrix=False, tf_gene=None,
                 sga_gene_list=None, embed_pretrain_gene2vec=True):
        super(ResGit, self).__init__()
        self.sga_size = sga_size
        self.out_feat_size = out_feat_size
        self.can_size = can_size
        self.embedding_dim = embedding_dim
        self.embedding_dim_last = embedding_dim_last
        self.n_head = n_head
        self.num_attn_blocks = num_attn_blocks
        self.using_cancer_type = using_cancer_type
        self.using_tf_gene_matrix = using_tf_gene_matrix
        self.tf_gene = tf_gene
        self.sga_gene_list = sga_gene_list
        self.embed_pretrain_gene2vec = embed_pretrain_gene2vec

        # Cancer type embeddings
        self.layer_can_emb = nn.Embedding(num_embeddings=can_size, embedding_dim=embedding_dim)

        # Attention and hidden layers
        self.SGA_blocks = nn.ModuleList()
        self.hiddenLayers = nn.ModuleList()

        # First attention block
        self.SGA_blocks.append(SGASelfAttn(sga_gene_list, sga_size, embedding_dim, n_head, embed_pretrain_gene2vec))

        # Intermediate attention blocks
        for i in range(1, num_attn_blocks - 1):
            self.SGA_blocks.append(SGASelfAttn(sga_gene_list, sga_size, embedding_dim, n_head, embed_pretrain_gene2vec))
            self.hiddenLayers.append(nn.Linear(embedding_dim, embedding_dim, bias=False))

        # Last attention block
        self.SGA_blocks.append(SGASelfAttn(sga_gene_list, sga_size, embedding_dim_last, n_head, embed_pretrain_gene2vec))
        self.hiddenLayers.append(nn.Linear(embedding_dim, embedding_dim_last, bias=False))

        # Final output layer
        if using_tf_gene_matrix:
            self.layer_final = nn.Linear(self.tf_gene.shape[0], self.out_feat_size, bias=False)
            self.mask_value = torch.FloatTensor(self.tf_gene.T).to(self.layer_final.weight.device)  # Ensure same device
            
            self.layer_final.weight.data = self.layer_final.weight.data * self.mask_value

            self.layer_final.weight.register_hook(lambda grad: grad.mul_(self.mask_value.to(grad.device)))
            self.hiddenLayers.append(self.layer_final)
        else:
            self.hiddenLayers.append(nn.Linear(self.embedding_dim_last, self.out_feat_size, bias=False))
        self.activationF = nn.ReLU()

    def forward(self, sga_index, can_index, store_hidden_layers=True):
        batch_attn_wts = {}
        hidden_layer_outs = {}

        # Initial attention block
        curr_hidden, attn_wts = self.SGA_blocks[0](sga_index)
        if self.using_cancer_type:
            curr_hidden += self.layer_can_emb(can_index)
        curr_hidden = self.activationF(curr_hidden)

        batch_attn_wts[0] = attn_wts
        hidden_layer_outs[0] = curr_hidden

        # Intermediate attention blocks
        for i in range(1, len(self.hiddenLayers)):
            sga_emb_after_attn, attn_wts = self.SGA_blocks[i](sga_index)
            curr_hidden = sga_emb_after_attn + self.hiddenLayers[i - 1](curr_hidden)
            if i < len(self.hiddenLayers) - 1:
                curr_hidden = self.activationF(curr_hidden)
            else:
                curr_hidden = torch.sigmoid(curr_hidden)

            if store_hidden_layers:
                hidden_layer_outs[i] = curr_hidden
            batch_attn_wts[i] = attn_wts

        # Final output
        preds = self.hiddenLayers[-1](curr_hidden)
        return preds, batch_attn_wts, hidden_layer_outs


# Training

In [ ]:
# Prepare inputs for GIT models
# sga_df = sga_df.applymap(lambda x: x + 1 if x != 0 else 0)
X, y = sga_df, deg_df
kf = KFold(n_splits=n_split, shuffle=True)

# Cross-validation loop
for fold_idx, ((tcga_train_idx, tcga_test_idx), (gdsc_train_idx, gdsc_test_idx)) in \
    enumerate(zip(kf.split(X.iloc[:len(tcga_sga_df)]), kf.split(X.iloc[len(tcga_sga_df):]))):
    print(f"Fold {fold_idx}")
    gdsc_train_idx = [i+tcga_sga_df.shape[0] for i in gdsc_train_idx]
    gdsc_test_idx = [j+tcga_sga_df.shape[0] for j in gdsc_test_idx]
    # Prepare training and testing sets
    X_train = X.iloc[list(tcga_train_idx) + gdsc_train_idx]
    X_test = X.iloc[list(tcga_test_idx) + gdsc_test_idx]
    print(X_train)
    can_train = can_df.loc[X_train.index].iloc[:, -1]  # cancer type indicies for training set
    can_test = can_df.loc[X_test.index].iloc[:, -1] # cancer type indicies for testing set
    
    y_train = y.loc[X_train.index]
    y_test = y.loc[X_test.index]

    if scale_y:
        y_train = scale(y_train, axis=1)
        y_test = scale(y_test, axis=1)
    y_train = torch.FloatTensor(y_train)
    y_test = torch.FloatTensor(y_test)

    X_train = torch.LongTensor(X_train.values)
    X_test = torch.LongTensor(X_test.values)
    can_train = torch.LongTensor(can_train.values.astype(int)).unsqueeze(dim=1)
    can_test = torch.LongTensor(can_test.values.astype(int)).unsqueeze(dim=1)
    
    X_train = torch.cat((X_train, can_train), dim=1)
    X_test = torch.cat((X_test, can_test), dim=1)

    train_set = torch.utils.data.TensorDataset(X_train, y_train)
    test_set = torch.utils.data.TensorDataset(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
    
    tf_gene_val = torch.FloatTensor(tf_gene_df.values)

    # Initialize the model
    model = ResGit(
        out_feat_size=y_train.shape[1],
        sga_size=sga_df.shape[1],
        embedding_dim=embedding_size,
        embedding_dim_last=embedding_size_last,
        can_size=can_size,
        n_head=n_head,
        num_attn_blocks=num_blocks,
        using_cancer_type=using_cancer_type,
        using_tf_gene_matrix=using_tf_gene_matrix,
        tf_gene=tf_gene_val,
        sga_gene_list=sga_df.columns.tolist(),
        embed_pretrain_gene2vec=embed_pretrain_gene2vec
    )
    model = model.to(device)

    # Apply pruning if enabled
    if hidden_weights_pruning:
        for ind, (_, module) in enumerate(model.hiddenLayers.named_modules()):
            if isinstance(module, nn.Linear) and ind < len(model.hiddenLayers) - 1:
                prune.l1_unstructured(module, name="weight", amount=hidden_weights_pruning_ratio)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, "min")
    criterion = nn.MSELoss()

    # Training loop
    best_model = None
    min_loss = float("inf")
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in tqdm(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs, _, _ = model(X_batch[:, :-1], X_batch[:, -1])
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        print(f"Epoch {epoch}, Training Loss: {train_loss}")

        # Evaluation on test set
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in tqdm(test_loader):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs, _, _ = model(X_batch[:, :-1], X_batch[:, -1])
                loss = criterion(outputs, y_batch)
                test_loss += loss.item()
        test_loss /= len(test_loader)
        print(f"Test Loss: {test_loss}")

        # Early stopping
        if test_loss < min_loss:
            min_loss = test_loss
            best_model = copy.deepcopy(model.state_dict())
        else:
            scheduler.step(train_loss)


Fold 0
                 B2M  SNX14  TET1  BCL2L1  HDAC1  RICTOR  GNGT2  MUC2  ROS1  \
TCGA-V1-A9ZK-01  0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
TCGA-55-8205-01  0.0    2.0   3.0     0.0    0.0     0.0    0.0   0.0   0.0   
TCGA-DR-A0ZM-01  1.0    0.0   0.0     0.0    0.0     6.0    0.0   0.0   0.0   
TCGA-HM-A3JK-01  0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
TCGA-KS-A4I1-01  0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
...              ...    ...   ...     ...    ...     ...    ...   ...   ...   
1298349          0.0    0.0   0.0     0.0    0.0     6.0    0.0   8.0   0.0   
909779           0.0    0.0   0.0     0.0    0.0     6.0    0.0   0.0   0.0   
1240193          0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   
1240168          0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   9.0   
753583           0.0    0.0   0.0     0.0    0.0     0.0    0.0   0.0   0.0   

                  ALK  ...   AR   EPHA6  LET

100%|██████████| 192/192 [00:07<00:00, 26.03it/s]


Epoch 0, Training Loss: 0.4962653693122168


100%|██████████| 22/22 [00:00<00:00, 84.29it/s]


Test Loss: 0.6325337236577814


...

Test Loss: 0.30693249675360595


100%|██████████| 192/192 [00:07<00:00, 27.28it/s]


Epoch 59, Training Loss: 0.16458255906278887


100%|██████████| 22/22 [00:00<00:00, 87.45it/s]

Test Loss: 0.30784677781841974


# Testing

In [ ]:
# Load best model and evaluate
model.load_state_dict(best_model)
model.eval()

for phase, loader in [("train", train_loader), ("test", test_loader)]:
    predictions, targets = [], []
    for X_batch, y_batch in loader:
        with torch.no_grad():
            X_batch = X_batch.to(device)  
            outputs, _, _ = model(X_batch[:, :-1], X_batch[:, -1]) 
            predictions.append(outputs.cpu().numpy()) 
            targets.append(y_batch.numpy()) 

    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)

    np.save(f"{phase}_predictions.npy", predictions)
    np.save(f"{phase}_targets.npy", targets)

    if phase == 'test':  
        from sklearn.metrics import mean_squared_error, mean_absolute_error

        mse = mean_squared_error(targets, predictions)
        mae = mean_absolute_error(targets, predictions)

        print(f"{phase} MSE: {mse:.4f}")
        print(f"{phase} MAE: {mae:.4f}")


test MSE: 0.2992
test MAE: 0.4090
